<a href="https://colab.research.google.com/github/9643Rafia/math-riddler-generator/blob/main/q1_a04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install transformers datasets torch accelerate einops

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import gc

In [3]:
# Free up memory
torch.cuda.empty_cache()
gc.collect()

90

In [4]:
# ✅ Load a smaller model to avoid memory errors
model_name = "Qwen/Qwen1.5-0.5B-Chat"

In [5]:
# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True,
    device_map="auto"
)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [7]:
# ✅ Load dataset (Ensure the file is uploaded)
dataset_path = "math_riddles.csv"  # Change this if needed
dataset = load_dataset("csv", data_files=dataset_path)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def tokenize_function(examples):
    inputs = [r + " Answer: " + s for r, s in zip(examples["Riddle"], examples["Solution"])]
    tokenized_inputs = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=256
    )

    # ✅ Labels should be the same as input_ids for causal LM
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()

    return tokenized_inputs

In [9]:
# ✅ Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [10]:
# ✅ Train-validation split
dataset_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    "train": dataset_split["train"],
    "validation": dataset_split["test"]
})

In [11]:
# ✅ Define training arguments with reduced memory load
training_args = TrainingArguments(
    output_dir="./qwen-finetune",
    per_device_train_batch_size=1,  # Lower batch size to avoid memory errors
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,  # Reduce if running out of memory
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=7,  # ⬆ Increase if needed
    save_total_limit=2,
    bf16=torch.cuda.is_bf16_supported(),  # Use bfloat16 if supported
    fp16=not torch.cuda.is_bf16_supported(),  # Otherwise, use float16
    optim="adamw_torch",
    report_to="none",
    logging_dir="./logs",
    logging_steps=10
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# ✅ Enable gradient checkpointing to reduce memory usage
model.gradient_checkpointing_enable()

In [13]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

20

In [14]:
!pip install flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 47.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl size=187831595 sha256=58853b28a5a926cae14402bfd8d4d93a45ebf8f9e79533f37ab09d0d77a99c05
  Stored in directory: /root/.cache/pip/wheels/3d/88/d8/284b89f56af7d5bf366b10d6b8e251ac8a7c7bf3f04203fb4f
Successfully built flash-attn


In [15]:
torch.backends.cuda.enable_flash_sdp(True)

In [16]:
# ✅ Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer
)

<ipython-input-16-ade8a88f2a32>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
# ✅ Start training
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,No log,0.198578
2,No log,0.180299
3,1.394400,0.161326
4,1.394400,0.152343
5,0.101400,0.150748


TrainOutput(global_step=21, training_loss=0.7158167312542597, metrics={'train_runtime': 499.1021, 'train_samples_per_second': 0.379, 'train_steps_per_second': 0.042, 'total_flos': 67740595912704.0, 'train_loss': 0.7158167312542597, 'epoch': 5.296296296296296})

In [18]:
# Save model and tokenizer
model.save_pretrained("./qwen-math-riddles")
tokenizer.save_pretrained("./qwen-math-riddles")


('./qwen-math-riddles/tokenizer_config.json',
 './qwen-math-riddles/special_tokens_map.json',
 './qwen-math-riddles/vocab.json',
 './qwen-math-riddles/merges.txt',
 './qwen-math-riddles/added_tokens.json',
 './qwen-math-riddles/tokenizer.json')

In [35]:
from shutil import make_archive
make_archive("qwen-math-riddles", 'zip', "/content/qwen-math-riddles")
from google.colab import files
files.download("qwen-math-riddles.zip")

RuntimeError: File size too large, try using force_zip64

In [20]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.3 MB/s eta 0:00:00


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from docx import Document

# Load fine-tuned model
model_path = "./qwen-math-riddles"  # Update if needed
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate riddles
def generate_riddle(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    output = model.generate(input_ids, max_length=150, temperature=0.7, top_k=50, top_p=0.9)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Generate 5 riddles
riddles = []
for i in range(5):
    prompt = "Here is a math riddle:\n"
    riddle = generate_riddle(prompt)
    riddles.append(riddle)
    print(f"Riddle {i+1}: {riddle}\n")

# Select the best riddles manually or use simple filtering (e.g., removing duplicates)
selected_riddles = riddles[:3]  # Modify based on testing

# Save the best riddles in a Word document
doc = Document()
doc.add_heading('Generated Math Riddles', level=1)

for idx, riddle in enumerate(selected_riddles, 1):
    doc.add_paragraph(f'Riddle {idx}: {riddle}\n')

# Save the file
doc.save("best_math_riddles.docx")
print("Best riddles saved to 'best_math_riddles.docx'")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Riddle 1: Here is a math riddle:
I am the sum of my digits, and I multiply by 10. What am I?
Answer: 36

Riddle 2: Here is a math riddle:
What number am I? Answer: 3 and 5

Riddle 3: Here is a math riddle:
There are 10 numbers in a row, and each number is three times the sum of the two numbers before it. What number am I? Answer: 30

Riddle 4: Here is a math riddle:
What number am I? Answer: 4

Riddle 5: Here is a math riddle:
I am the product of my digits, and multiply by 24. What number am I? Answer: 36

Best riddles saved to 'best_math_riddles.docx'


In [22]:
!pip install streamlit flask flask-cloudflared

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.3 MB/s eta 0:00:00


In [25]:
!streamlit run app.py & sleep 3 && npx cloudflared tunnel --url http://localhost:8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.215.123:8501

⠙⠹⠸⠼⠴2025-03-14T17:23:40Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-03-14T17:23:40Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-03-14T17:23:43Z INF +--------------------------------------------------------------------------------------------+
2025-03-14T17:23:43Z